In [17]:
%load_ext autoreload
%autoreload 2

import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import *
import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, os.environ["PROJECT_DIR"] + "/estimations/basecamp")
os.chdir(os.environ["PROJECT_DIR"] + "/estimations/basecamp")
from moments import get_moments

In [9]:
fname = os.environ["PROJECT_DIR"] + "/resources/soepcore_struct_prep.dta"
df_obs = pd.read_stata(fname, convert_categoricals=False)
df_obs = df_alignment(df_obs)

params_start = pd.read_pickle("start.soepy.pkl")
fname  = "resources/model_spec_init.yml"
df_sim = df_alignment(soepy.simulate(params_start, fname))

In [43]:
_ = get_moments(df_obs)
df_obs_int = pd.read_pickle("df-debug.pkl")

In [44]:
_ = get_moments(df_sim)
df_sim_int = pd.read_pickle("df-debug.pkl")

In [46]:
df_sim_int

Value
Period Education_Level Choice          
0      High            Home    0.108428
                       Part    0.161952
                       Full    0.729621
       Medium          Home    0.221378
                       Part    0.127542
                       Full    0.651080
       Low             Home    0.564919
                       Part    0.036127
                       Full    0.398954
1      High            Home    0.124101
                       Part    0.201479
                       Full    0.674421
       Medium          Home    0.223487
                       Part    0.155398
                       Full    0.621115
       Low             Home    0.575513
                       Part    0.053035
                       Full    0.371452
2      High            Home    0.149433
                       Part    0.224544
                       Full    0.626023
       Medium          Home    0.264237
                       Part    0.160412
                       Full    0.575351
       Low             Home    0.589909
                       Part    0.068994
                       Full    0.341097
3      High            Home    0.178314
                       Part    0.246821
                       Full    0.574864
       Medium          Home    0.305384
                       Part    0.165705
                       Full    0.528911
       Low             Home    0.601114
                       Part    0.081760
                       Full    0.317126
4      High            Home    0.205914
                       Part    0.267225
                       Full    0.526861
       Medium          Home    0.338653
                       Part    0.182817
                       Full    0.478531
       Low             Home    0.614423
                       Part    0.101929
                       Full    0.283648
5      High            Home    0.260522
                       Part    0.266437
                       Full    0.473041
       Medium          Home    0.367902
                       Part    0.199491
                       Full    0.432608
       Low             Home    0.619109
                       Part    0.123998
                       Full    0.256893
6      High            Home    0.305865
                       Part    0.280532
                       Full    0.413603
       Medium          Home    0.388396
                       Part    0.223845
                       Full    0.387759
       Low             Home    0.626307
                       Part    0.145661
                       Full    0.228032
7      High            Home    0.324199
                       Part    0.298472
                       Full    0.377329
       Medium          Home    0.407975
                       Part    0.245493
                       Full    0.346532
       Low             Home    0.631944
                       Part    0.155371
                       Full    0.212685
8      High            Home    0.373189
                       Part    0.303795
                       Full    0.323016
       Medium          Home    0.428191
                       Part    0.258387
                       Full    0.313423
       Low             Home    0.621961
                       Part    0.163045
                       Full    0.214994
9      High            Home    0.376639
                       Part    0.333366
                       Full    0.289995
       Medium          Home    0.436746
                       Part    0.283537
                       Full    0.279717
       Low             Home    0.625628
                       Part    0.162841
                       Full    0.211531
10     High            Home    0.401479
                       Part    0.321636
                       Full    0.276885
       Medium          Home    0.445461
                       Part    0.294719
                       Full    0.259819
       Low             Home    0.620128
                       Part    0.161347
                       Full    0.2